In [4]:
import pandas as pd
import numpy as np

In [5]:
recipes = pd.read_csv('../dataset/RAW_recipes.csv')
interactions = pd.read_csv('../dataset/RAW_interactions.csv')

In [6]:
preprocessed_df = pd.read_csv('../dataset/preprocessed_data.csv')
preprocessed_df.head()

,user_id,recipe_id,date,rating,review,minutes,contributor_id,submitted,tags,n_steps,...,sugar,sodium,protein,saturated_fat,carbohydrates,food_types,neg,neu,pos,compound
0,492,20636,2002-12-01,4,this worked very well and is easy. i used not ...,20,56824,2002-10-27,"30-minutes-or-less, time-to-make, course, main...",5,...,39.0,5.0,4.0,11.0,5.0,Healthy,0.0,0.598,0.402,0.8553
1,8204,64566,2005-09-02,4,very good,40,166019,2005-08-24,"60-minutes-or-less, time-to-make, main-ingredi...",10,...,40.0,37.0,78.0,4.0,10.0,Non-veg,0.0,0.238,0.762,0.4927
2,28657,64566,2005-12-22,5,better than the real,40,166019,2005-08-24,"60-minutes-or-less, time-to-make, main-ingredi...",10,...,40.0,37.0,78.0,4.0,10.0,Non-veg,0.0,0.508,0.492,0.4404
3,36365,64566,2006-09-26,5,absolutely awesome i was speechless when i tri...,40,166019,2005-08-24,"60-minutes-or-less, time-to-make, main-ingredi...",10,...,40.0,37.0,78.0,4.0,10.0,Non-veg,0.0,0.883,0.117,0.6590
4,20197,64566,2007-03-09,5,these taste absolutely wonderful my son in law...,40,166019,2005-08-24,"60-minutes-or-less, time-to-make, main-ingredi...",10,...,40.0,37.0,78.0,4.0,10.0,Non-veg,0.0,0.675,0.325,0.8908


In [7]:
preprocessed_df = preprocessed_df[['user_id', 'recipe_id']]
pivot = preprocessed_df.pivot_table(index='user_id', columns='recipe_id', aggfunc=len, fill_value=0)
array = pivot.to_numpy()

/var/folders/mg/gscfdh1x5zxcpw79_4mrfny00000gn/T/ipykernel_4163/2377383315.py:2: PerformanceWarning: The following operation may generate 38782857123 cells in the resulting pandas object.
  pivot = preprocessed_df.pivot_table(index='user_id', columns='recipe_id', aggfunc=len, fill_value=0)


: 

: 

In [ ]:
""" Non-negative matrix factorization (tensorflow)"""
# Author: Eesung Kim <eesungk@gmail.com>

import numpy as np
import tensorflow as tf

class NMF:
    """Compute Non-negative Matrix Factorization (NMF)"""
    def __init__(self, max_iter=200, learning_rate=0.01,display_step=10, optimizer='mu', initW=False):

        self.max_iter = max_iter
        self.learning_rate= learning_rate
        self.display_step = display_step
        self.optimizer = optimizer

    def NMF(self, X, r_components, learning_rate, max_iter, display_step, optimizer, initW, givenW ):
        m,n=np.shape(X)
        tf.reset_default_graph()
        V = tf.placeholder(tf.float32) 

        initializer = tf.random_uniform_initializer(0,1)
        if initW is False:
            W =  tf.get_variable(name="W", shape=[m, r_components], initializer=initializer)
            H =  tf.get_variable("H", [r_components, n], initializer=initializer)
        else:
            W =  tf.constant(givenW, shape=[m, r_components], name="W")
            H =  tf.get_variable("H", [r_components, n], initializer=initializer)

        WH =tf.matmul(W, H)
        cost = tf.reduce_sum(tf.square(V - WH))
        
        if optimizer=='mu':
            """Compute Non-negative Matrix Factorization with Multiplicative Update"""
            Wt = tf.transpose(W)
            H_new = H * tf.matmul(Wt, V) / tf.matmul(tf.matmul(Wt, W), H)
            H_update = H.assign(H_new)

            if initW is False:
                Ht = tf.transpose(H)
                W_new = W * tf.matmul(V, Ht)/ tf.matmul(W, tf.matmul(H, Ht))
                W_update = W.assign(W_new)

        elif optimizer=='pg':
            """optimization; Projected Gradient method """
            dW, dH = tf.gradients(xs=[W, H], ys=cost)
            H_update_ = H.assign(H - learning_rate * dH)
            H_update = tf.where(tf.less(H_update_, 0), tf.zeros_like(H_update_), H_update_)

            if initW is False:
                W_update_ = W.assign(W - learning_rate * dW)
                W_update = tf.where(tf.less(W_update_, 0), tf.zeros_like(W_update_), W_update_)

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for idx in range(max_iter):
                if initW is False:
                    W=sess.run(W_update, feed_dict={V:X})
                    H=sess.run(H_update, feed_dict={V:X})
                else:
                    H=sess.run(H_update, feed_dict={V:X})

                if (idx % display_step) == 0:
                    costValue = sess.run(cost,feed_dict={V:X})
                    print("|Epoch:","{:4d}".format(idx), " Cost=","{:.3f}".format(costValue))

        return W, H

    def fit_transform(self, X,r_components, initW, givenW):
        """Transform input data to W, H matrices which are the non-negative matrices."""
        W, H =  self.NMF(X=X, r_components = r_components, learning_rate=self.learning_rate, 
                    max_iter = self.max_iter, display_step = self.display_step, 
                    optimizer=self.optimizer, initW=initW, givenW=givenW  )
        return W, H

    def inverse_transform(self, W, H):
        """Transform data back to its original space."""
        return np.matmul(W,H)

def main():
    V = np.array([[1, 1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])
    model = NMF(max_iter=200,learning_rate=0.01,display_step=10, optimizer='mu')
    W, H = model.fit_transform(V, r_components=2, initW=False, givenW=0)
    print(W)
    print(H)
    print(V)
    print(model.inverse_transform(W, H))

In [ ]:
model = NMF(max_iter=200,learning_rate=0.01,display_step=10, optimizer='mu')
W, H = model.fit_transform(array, r_components=2, initW=False, givenW=0)

In [ ]:
print(W)
print(H)
print(array)

In [ ]:
from sklearn.decomposition import NMF
n_components = 10

# create an instance of the NMF model
nmf_model = NMF(n_components=n_components)

# fit the NMF model to the data
nmf_model.fit(array)

# extract the factor matrices from the NMF model
W = nmf_model.transform(array)
H = nmf_model.components_

# print the shapes of the factor matrices
print("Shape of W matrix: ", W.shape)
print("Shape of H matrix: ", H.shape)

In [ ]:
# Evaluate the score of NMF 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

def evaluate_score(array, W, H):
    score = 0
    for i in range(len(array)):
        for j in range(len(array[0])):
            score += (array[i][j] - np.dot(W[i], H[:,j])) ** 2
    return score

print("MSE: ", evaluate_score(array, W, H))
print("MAE: ", mean_absolute_error(array, np.dot(W, H)))
print("R2: ", r2_score(array, np.dot(W, H)))
